## Goal: intersect PLSs w/ TSSs called by LAPA for each sample

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [61]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [74]:
df = load_meta()
df = df.loc[~df['sample'].str.contains('_')]
df['lab_sample'] = df['lab_number_sample'].astype(str)+'_'+\
                      df['lab_sampleid'].astype(str)+'_'+\
                      df['cell_line_id'].astype(str)
df = df.loc[df.mixed_samples==False]
df = df[['cell_line_id', 'sample', 'hapmap_DNA_ID', 'lab_sample']].drop_duplicates()

In [75]:
pls_file = proc_cfg(config['ref']['ccre']['pls_bed'], od)
pls = pr.read_bed(pls_file)

In [76]:
bed_files = expand(proc_cfg(config['lr']['lapa']['sample_tss'], od),
                   sample=df['sample'].tolist())

In [77]:
f = bed_files[0]
f = '../data/lapa/sample/YRI7.bed'
tss = pr.read_bed(f)

tss = tss.df
tss.rename({'ItemRGB': 'tpm'}, axis=1, inplace=True)
tss['rid'] = tss['Chromosome'].astype(str)+'_'+\
             tss['Start'].astype(str)+'_'+\
             tss['End'].astype(str)
tss = tss[['Chromosome', 'Start', 'End', 'tpm', 'rid']]
tss = tss.drop_duplicates()
l1 = len(tss.index)

In [79]:
# assert len(tss) == len(tss.rid.unique())
tss.loc[tss.rid.duplicated(keep=False)].sort_values(by='rid').head()


,Chromosome,Start,End,Strand,tpm,rid
26148,chrM,14490,14491,+,4.03,chrM_14490_14491
26151,chrM,14490,14491,-,4.03,chrM_14490_14491


In [67]:
tss = pr.PyRanges(tss)
tss = tss.join(pls,
               strandedness=None,
               how='left')

In [68]:
# tss.loc[tss.Start_b != -1]

In [69]:
# deduplicate entries from original TSSs by whether the intersect
# at least one ccre 
tss = tss.df
tss['int_pls'] = False
tss.loc[tss.Start_b != -1, 'int_pls'] = True
gb_cols = ['Chromosome', 'Start', 'End', 'Strand', 'tpm', 'rid']
keep_cols = gb_cols+['int_pls']
tss = tss[keep_cols].groupby(gb_cols, observed=True).max().reset_index()

In [70]:
# make sure there are no dupes
assert len(tss) == len(tss.rid.unique())
l2 = len(tss.index)
assert l1 == l2

,Chromosome,Start,End,Strand,tpm,rid,int_pls
0,GL000194.1,53585,53595,+,6.18,GL000194.1_53585_53595,False
1,GL000194.1,61655,61675,+,5.47,GL000194.1_61655_61675,False
2,GL000194.1,62358,62365,+,2.97,GL000194.1_62358_62365,False
3,GL000194.1,62628,62636,+,3.99,GL000194.1_62628_62636,False
4,GL000194.1,62665,62676,+,8.21,GL000194.1_62665_62676,False
